<a href="https://colab.research.google.com/github/Youssef1Moustafa/Olymbic_prediction/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#!pip install lightgbm

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from imblearn.over_sampling import SMOTE

In [13]:
df = pd.read_csv('olymbic.csv')
df.head()

,Sex,NOC,Year,Season,City,Sport,Event,Medal,Country,won_or_not,Gold_Count,Silver_Count,Bronze_Count,Total_Medals,Total_Medals_country
0,1,42,1992,0,4,12,164,2,202,0,0.0,0.0,0.0,0,1218
1,1,42,2012,0,8,43,356,2,202,0,0.0,0.0,0.0,0,1218
2,1,56,1920,0,1,36,315,2,283,0,0.0,0.0,0.0,0,666
3,1,56,1900,0,15,71,822,1,288,1,200.0,0.0,0.0,1,666
4,0,148,1932,0,9,8,131,2,713,0,0.0,0.0,0.0,0,1108


In [14]:
df = df.drop(['Gold_Count','Silver_Count','Bronze_Count','Total_Medals','Total_Medals_country'], axis=1)

In [15]:
from sklearn.model_selection import train_test_split

#selected_features = positive_corr_delivery_type['Feature'].tolist()
#X = df[selected_features]
X = df.drop(['won_or_not'], axis=1)
y = df['won_or_not']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [16]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [17]:
print(f"Resampled class distribution: \n{y_train_resampled.value_counts()}")

Resampled class distribution: 
won_or_not
0    106954
1    106954
Name: count, dtype: int64


In [18]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)
y_pred = model.predict(X_test)

In [19]:
print("Classification Report:\n", classification_report(y_test, y_pred))
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"ROC-AUC Score: {roc_auc:.2f}")

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99    106793
           1       0.95      1.00      0.97     19490

    accuracy                           0.99    126283
   macro avg       0.97      1.00      0.98    126283
weighted avg       0.99      0.99      0.99    126283

ROC-AUC Score: 1.00


In [20]:
# Step 5: Define a function to train and evaluate models
def train_and_evaluate_model(model, model_name):
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    print(f"--- {model_name} ---")
    print("Classification Report:\n", classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f"ROC-AUC Score: {roc_auc:.2f}\n")

# Step 6: Train and evaluate different models
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "Random Forest": RandomForestClassifier(class_weight='balanced', random_state=42),
    #"Support Vector Machine": SVC(class_weight='balanced', probability=True),
    "XGBoost": xgb.XGBClassifier(scale_pos_weight=(len(y) - sum(y)) / sum(y), random_state=42),
    "LightGBM": lgb.LGBMClassifier(class_weight='balanced', random_state=42)
}

for model_name, model in models.items():
    train_and_evaluate_model(model, model_name)

--- Logistic Regression ---
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94    106793
           1       0.65      0.67      0.66     19490

    accuracy                           0.89    126283
   macro avg       0.80      0.80      0.80    126283
weighted avg       0.90      0.89      0.89    126283

ROC-AUC Score: 0.67

--- Decision Tree ---
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99    106793
           1       0.92      1.00      0.96     19490

    accuracy                           0.99    126283
   macro avg       0.96      0.99      0.97    126283
weighted avg       0.99      0.99      0.99    126283

ROC-AUC Score: 0.99

--- Random Forest ---
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99    106793
           1       0.95      1.00      0.97     194

# Importance features

In [21]:
X = df[['NOC','Country','Sport','Event']]
y = df['won_or_not']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [22]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [23]:
print(f"Resampled class distribution: \n{y_train_resampled.value_counts()}")

Resampled class distribution: 
won_or_not
0    106954
1    106954
Name: count, dtype: int64


In [24]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)
y_pred = model.predict(X_test)

In [25]:
print("Classification Report:\n", classification_report(y_test, y_pred))
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"ROC-AUC Score: {roc_auc:.2f}")

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.82      0.88    106793
           1       0.43      0.75      0.55     19490

    accuracy                           0.81    126283
   macro avg       0.69      0.79      0.71    126283
weighted avg       0.87      0.81      0.83    126283

ROC-AUC Score: 0.86


In [26]:
# Step 5: Define a function to train and evaluate models
def train_and_evaluate_model(model, model_name):
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    print(f"--- {model_name} ---")
    print("Classification Report:\n", classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f"ROC-AUC Score: {roc_auc:.2f}\n")

# Step 6: Train and evaluate different models
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "Random Forest": RandomForestClassifier(class_weight='balanced', random_state=42),
    "XGBoost": xgb.XGBClassifier(scale_pos_weight=(len(y) - sum(y)) / sum(y), random_state=42),
    "LightGBM": lgb.LGBMClassifier(class_weight='balanced', random_state=42)
}

for model_name, model in models.items():
    train_and_evaluate_model(model, model_name)

--- Logistic Regression ---
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.54      0.67    106793
           1       0.18      0.56      0.28     19490

    accuracy                           0.55    126283
   macro avg       0.53      0.55      0.47    126283
weighted avg       0.76      0.55      0.61    126283

ROC-AUC Score: 0.57

--- Decision Tree ---
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.82      0.88    106793
           1       0.43      0.74      0.55     19490

    accuracy                           0.81    126283
   macro avg       0.69      0.78      0.71    126283
weighted avg       0.87      0.81      0.83    126283

ROC-AUC Score: 0.85

--- Random Forest ---
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.82      0.88    106793
           1       0.43      0.75      0.55     194

In [27]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Step 4: Apply K-Fold Cross-Validation and store accuracies
accuracies = []

for train_index, test_index in kf.split(X, y):
    # Split into train and test for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE on the training data to handle imbalance
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train the RandomForest model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_resampled, y_train_resampled)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy for this fold
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"Accuracy for this fold: {acc:.4f}")

Accuracy for this fold: 0.8081
Accuracy for this fold: 0.8034
Accuracy for this fold: 0.8102
Accuracy for this fold: 0.8087
Accuracy for this fold: 0.8081


In [28]:
import numpy as np
# Step 5: Calculate the mean accuracy across all folds
mean_accuracy = np.mean(accuracies)
print(f"\nMean Accuracy across all folds: {mean_accuracy:.4f}")


Mean Accuracy across all folds: 0.8077


In [29]:
import joblib
# Initialize StratifiedKFold and SMOTE
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
smote = SMOTE(random_state=42)

accuracies = []
best_model = None
best_accuracy = 0

# K-Fold Cross-Validation with SMOTE and RandomForest
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to the training data
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train the RandomForest model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_resampled, y_train_resampled)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"Accuracy for this fold: {acc:.4f}")

    # Save the best model
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model

Accuracy for this fold: 0.8081
Accuracy for this fold: 0.8034
Accuracy for this fold: 0.8102
Accuracy for this fold: 0.8087
Accuracy for this fold: 0.8081


In [30]:
# Calculate mean accuracy
import joblib

mean_accuracy = np.mean(accuracies)
print(f"\nMean Accuracy: {mean_accuracy:.4f}")

# Save the best model to a .pkl file
joblib.dump(best_model, 'best_model.pkl')
print("Model saved as 'best_model.pkl'.")

# Load the model (Example)
loaded_model = joblib.load('best_model.pkl')
print("Model loaded successfully.")


Mean Accuracy: 0.8077
Model saved as 'best_model.pkl'.
Model loaded successfully.
